<a href="https://colab.research.google.com/github/Xu-Ruixin/Multilingual-Phoneme-Recognition/blob/main/CS2NO_Phoneme_Recognition_(Piloting).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sun Dec  3 19:09:01 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
%%capture
!pip install -U datasets
!pip install -U transformers[torch]
!pip install torchaudio
!pip install librosa
!pip install -U huggingface_hub
!pip install jiwer
!pip install accelerate -U
!pip install phonemizer
!apt install git-lfs
!sudo apt-get install festival espeak-ng mbrola
!apt search mbrola
!apt install git-lfs

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from datasets import load_dataset, load_metric, Audio

# common_voice_train = load_dataset("facebook/voxpopuli", "fi", split="train+validation")
common_voice_train = load_dataset("facebook/voxpopuli", "cs", split="validation")
common_voice_test = load_dataset("Sprakbanken/nb_samtale", split="test")
common_voice_test_2 = common_voice_test

In [ ]:
common_voice_train

Dataset({
    features: ['audio_id', 'language', 'audio', 'raw_text', 'normalized_text', 'gender', 'speaker_id', 'is_gold_transcript', 'accent'],
    num_rows: 1103
})

In [ ]:
common_voice_test

Dataset({
    features: ['source_file_id', 'segment_id', 'segment_order', 'duration', 'overlap_previous', 'overlap_next', 'speaker_id', 'gender', 'dialect', 'orthography', 'source_type', 'file_name', 'transcription', 'annotations', 'orthographic', 'verbatim', 'audio'],
    num_rows: 1195
})

In [ ]:
sentence1 = "transcription"
sentence2 = "normalized_text"
col_names1 = list(common_voice_test.features.keys())
col_names2 = list(common_voice_train.features.keys())
col_names1.remove("audio")
col_names2.remove("audio")
col_names1.remove(sentence1)
col_names2.remove(sentence2)
common_voice_train = common_voice_train.remove_columns(col_names2)
common_voice_test = common_voice_test.remove_columns(col_names1)

In [ ]:
common_voice_train

Dataset({
    features: ['audio', 'normalized_text'],
    num_rows: 1103
})

In [ ]:
common_voice_test

Dataset({
    features: ['transcription', 'audio'],
    num_rows: 1195
})

In [ ]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(common_voice_train, num_examples=1)

,audio,normalized_text
0,"{'path': '/root/.cache/huggingface/datasets/downloads/extracted/878bb93f50316d028ef5fbc7cc348b2f97af8adc6c081131daea7c82daa38113/dev_part_0/20171004-0900-PLENARY-14-cs_20171004-17:41:55_7.wav', 'array': [0.000274658203125, -9.1552734375e-05, -0.000152587890625, 0.0001220703125, 0.000274658203125, 0.000244140625, 0.0, 3.0517578125e-05, 0.000244140625, 0.000244140625, 0.0, -0.0001220703125, -3.0517578125e-05, 0.0001220703125, 0.00018310546875, 0.000335693359375, 0.00018310546875, -0.00018310546875, -0.000152587890625, 9.1552734375e-05, 0.0003662109375, 0.0003662109375, 0.000274658203125, 0.000274658203125, -3.0517578125e-05, -3.0517578125e-05, 0.0, 0.00018310546875, 0.00030517578125, 0.000244140625, 0.0001220703125, 0.000213623046875, 6.103515625e-05, -3.0517578125e-05, -9.1552734375e-05, 0.00018310546875, 0.00042724609375, 0.000213623046875, 0.000244140625, 0.0001220703125, 0.0001220703125, 0.000152587890625, 0.00018310546875, 6.103515625e-05, 0.0001220703125, 0.00030517578125, 0.000335693359375, 0.000213623046875, 0.0, 0.000274658203125, 0.000396728515625, 0.000244140625, 3.0517578125e-05, 9.1552734375e-05, 0.000152587890625, 3.0517578125e-05, 0.000152587890625, 0.000152587890625, 0.000152587890625, 9.1552734375e-05, -0.000274658203125, -0.000244140625, 0.00018310546875, 0.000244140625, 0.0, -9.1552734375e-05, -9.1552734375e-05, -3.0517578125e-05, 0.0001220703125, 0.000335693359375, 0.00042724609375, 0.000274658203125, -0.0001220703125, -0.0001220703125, 0.00018310546875, -0.00018310546875, -0.000335693359375, 0.000274658203125, 0.00048828125, 0.00030517578125, 0.00018310546875, 0.0003662109375, 0.00048828125, 0.00018310546875, -0.000152587890625, -0.00042724609375, -0.000213623046875, 0.00048828125, 0.0006103515625, 0.000457763671875, 0.00018310546875, 0.0, 0.000244140625, 6.103515625e-05, -0.0001220703125, 0.000213623046875, 0.0003662109375, 0.00042724609375, 0.00030517578125, 0.000335693359375, ...], 'sampling_rate': 16000}",identický úkol stojí před evropskou komisí a členskými státy.


In [ ]:
import re
chars_to_remove_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�\']'

def remove_special_characters_test(batch):
    batch[sentence1] = re.sub(chars_to_remove_regex, '', batch[sentence1]).lower()
    return batch

def remove_special_characters_train(batch):
    batch['normalized_text'] = re.sub(chars_to_remove_regex, '', batch['normalized_text']).lower()
    return batch


In [ ]:
common_voice_train = common_voice_train.map(remove_special_characters_train)
common_voice_test = common_voice_test.map(remove_special_characters_test)

In [ ]:
show_random_elements(common_voice_train, 1)

,audio,normalized_text
0,"{'path': '/root/.cache/huggingface/datasets/downloads/extracted/878bb93f50316d028ef5fbc7cc348b2f97af8adc6c081131daea7c82daa38113/dev_part_0/20171025-0900-PLENARY-13-cs_20171025-13:36:39_4.wav', 'array': [0.046966552734375, 0.092041015625, 0.112640380859375, 0.0799560546875, 0.07464599609375, 0.099151611328125, 0.1136474609375, 0.111175537109375, 0.106842041015625, 0.14813232421875, 0.19451904296875, 0.23663330078125, 0.241729736328125, 0.244293212890625, 0.23187255859375, 0.209442138671875, 0.08740234375, -0.088897705078125, -0.28460693359375, -0.444549560546875, -0.5196533203125, -0.528228759765625, -0.426239013671875, -0.350250244140625, -0.3187255859375, -0.35711669921875, -0.365692138671875, -0.324737548828125, -0.241058349609375, -0.119293212890625, 0.02374267578125, 0.163330078125, 0.2705078125, 0.3328857421875, 0.329803466796875, 0.2818603515625, 0.198974609375, 0.12384033203125, 0.083770751953125, 0.0806884765625, 0.114349365234375, 0.096923828125, 0.05169677734375, -0.005157470703125, -0.0546875, -0.11083984375, -0.204803466796875, -0.242584228515625, -0.224090576171875, -0.129547119140625, -0.034210205078125, 0.034881591796875, 0.048309326171875, 0.036376953125, 0.032745361328125, 0.05712890625, 0.11907958984375, 0.178863525390625, 0.211669921875, 0.195068359375, 0.141326904296875, 0.084930419921875, 0.022857666015625, -0.0164794921875, -0.002655029296875, 0.029388427734375, 0.051055908203125, 0.027984619140625, 0.0059814453125, -0.026031494140625, -0.050750732421875, -0.092681884765625, -0.108367919921875, -0.06781005859375, -0.00494384765625, 0.054534912109375, 0.055755615234375, 0.04296875, 0.034515380859375, 0.0399169921875, 0.059844970703125, 0.10009765625, 0.15264892578125, 0.190277099609375, 0.21551513671875, 0.207916259765625, 0.195465087890625, 0.180267333984375, 0.1697998046875, 0.165130615234375, 0.146270751953125, 0.109588623046875, 0.000213623046875, -0.162322998046875, -0.3424072265625, -0.47735595703125, -0.527984619140625, -0.475555419921875, -0.39434814453125, -0.32208251953125, ...], 'sampling_rate': 16000}",zajistí také poskytování přesných informací o osobách které překračují délku povoleného pobytu


In [ ]:
from transformers import Wav2Vec2Processor

# processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-lv-60-espeak-cv-ft")

In [ ]:
common_voice_train = common_voice_train.cast_column("audio", Audio(sampling_rate=16_000))
common_voice_test = common_voice_test.cast_column("audio", Audio(sampling_rate=16_000))

In [ ]:
common_voice_train[0]["audio"]

{'path': '/root/.cache/huggingface/datasets/downloads/extracted/878bb93f50316d028ef5fbc7cc348b2f97af8adc6c081131daea7c82daa38113/dev_part_0/20171115-0900-PLENARY-12-cs_20171115-13:38:25_3.wav',
 'array': array([-0.0010376 , -0.00476074, -0.00054932, ..., -0.00021362,
        -0.00018311, -0.0010376 ]),
 'sampling_rate': 16000}

In [ ]:
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(common_voice_train)-1)

print(common_voice_train[rand_int]["normalized_text"])
ipd.Audio(data=common_voice_train[rand_int]["audio"]["array"], autoplay=True, rate=16000)

já jsem si nedávno studovala dokumenty které se vztahovaly ke konferenci ve wannsee kde šlo o pokračování praxe norimberských zákonů kde šlo taky o sčítání lidí na rasovém původu


In [ ]:
rand_int = random.randint(0, len(common_voice_train)-1)

print("Target text:", common_voice_train[rand_int]["normalized_text"])
print("Input array shape:", common_voice_train[rand_int]["audio"]["array"].shape)
print("Sampling rate:", common_voice_train[rand_int]["audio"]["sampling_rate"])

Target text: v kompromisním textu také evropský parlament požaduje postupné ukončení používání palmového oleje a dalších rostlinných olejů jako složky biopaliv s preferencí od roku
Input array shape: (152639,)
Sampling rate: 16000


In [ ]:
def prepare_dataset_train(batch):
    audio = batch["audio"]

    # batched output is "un-batched"
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    batch["input_length"] = len(batch["input_values"])

    with processor.as_target_processor():
        batch["labels"] = processor(batch[sentence2]).input_ids

    return batch
def prepare_dataset_test(batch):
    audio = batch["audio"]

    # batched output is "un-batched"
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    batch["input_length"] = len(batch["input_values"])

    with processor.as_target_processor():
        batch["labels"] = processor(batch[sentence1]).input_ids

    return batch

In [ ]:
common_voice_train = common_voice_train.map(prepare_dataset_train, remove_columns=common_voice_train.column_names)
common_voice_test = common_voice_test.map(prepare_dataset_test, remove_columns=common_voice_test.column_names)

Map:   0%|          | 0/1103 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/1195 [00:00<?, ? examples/s]

In [ ]:
max_input_length_in_sec = 5.0
common_voice_train = common_voice_train.filter(lambda x: x < max_input_length_in_sec * processor.feature_extractor.sampling_rate, input_columns=["input_length"])

common_voice_test = common_voice_test.filter(lambda x: x < max_input_length_in_sec * processor.feature_extractor.sampling_rate, input_columns=["input_length"])

Filter:   0%|          | 0/1103 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1195 [00:00<?, ? examples/s]

In [ ]:
common_voice_train

Dataset({
    features: ['input_values', 'input_length', 'labels'],
    num_rows: 185
})

In [ ]:
common_voice_test

Dataset({
    features: ['input_values', 'input_length', 'labels'],
    num_rows: 549
})

In [ ]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [ ]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [ ]:
wer_metric = load_metric("wer")

<ipython-input-27-605de035dfd7>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  wer_metric = load_metric("wer")


In [ ]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [ ]:
from transformers import Wav2Vec2ForCTC

dropout_ratio = 0.2

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-lv-60-espeak-cv-ft",
    attention_dropout=dropout_ratio,
    hidden_dropout=dropout_ratio,
    feat_proj_dropout=dropout_ratio,
    mask_time_prob=0.05,
    layerdrop=0.0,
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer),
)

config.json:   0%|          | 0.00/1.86k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/wav2vec2-lv-60-espeak-cv-ft were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-lv-60-espeak-cv-ft and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0']
You should probably

In [ ]:
model.freeze_feature_extractor()

/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:1913: FutureWarning: The method `freeze_feature_extractor` is deprecated and will be removed in Transformers v5. Please use the equivalent `freeze_feature_encoder` method instead.
  warnings.warn(


In [ ]:
from transformers import TrainingArguments

repo_name = "cs2no-wav2vec2-large-xls-r-300m-cs-colab-phoneme"

training_args = TrainingArguments(
  output_dir=repo_name,
  group_by_length=True,
  per_device_train_batch_size=16,
  gradient_accumulation_steps=2,
  evaluation_strategy="steps",
  num_train_epochs=50,
  gradient_checkpointing=True,
  fp16=True,
  save_steps=100,
  eval_steps=100,
  logging_steps=100,
  learning_rate=1e-5,
  warmup_steps=500,
  save_total_limit=2,
  push_to_hub=True,
)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=common_voice_train,
    eval_dataset=common_voice_test,
    tokenizer=processor.feature_extractor,
)

To avoid inactivity: inspect -> console -> copy-paste

```javascript
function ConnectButton(){
    console.log("Connect pushed");
    document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click()
}
setInterval(ConnectButton,60000);
```

In [ ]:
trainer.evaluate()

/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


{'eval_loss': 5.79113245010376,
 'eval_wer': 0.7625567433006296,
 'eval_runtime': 26.76,
 'eval_samples_per_second': 20.516,
 'eval_steps_per_second': 2.578}

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss,Wer
100,3.820000,5.281853,0.733636
200,2.883400,4.942372,0.709108
300,2.538700,4.917425,0.706326


/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Tran

TrainOutput(global_step=300, training_loss=3.08071294148763, metrics={'train_runtime': 1084.2821, 'train_samples_per_second': 8.531, 'train_steps_per_second': 0.277, 'total_flos': 1.2149188697400325e+18, 'train_loss': 3.08071294148763, 'epoch': 50.0})

In [ ]:
trainer.evaluate()

/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


{'eval_loss': 4.91742467880249,
 'eval_wer': 0.7063259628056816,
 'eval_runtime': 22.7553,
 'eval_samples_per_second': 24.126,
 'eval_steps_per_second': 3.032,
 'epoch': 50.0}

In [ ]:
input_dict = processor(common_voice_test[0]["input_values"], return_tensors="pt", padding=True)
print(input_dict)

logits = model(input_dict.input_values.to("cuda")).logits
print(logits)

pred_ids = torch.argmax(logits, dim=-1)[0]
print(pred_ids)

print("Prediction:")
print(processor.decode(pred_ids))

print("\nReference:")
print(common_voice_test_2[0][sentence1].lower())

It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


{'input_values': tensor([[0.0132, 0.0069, 0.0082,  ..., 0.0195, 0.0217, 0.0263]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1]], dtype=torch.int32)}
tensor([[[ 12.8906, -17.9219, -17.5156,  ..., -16.5938, -18.0625, -17.2656],
         [ 12.9688, -17.8594, -17.4531,  ..., -16.5469, -18.0156, -17.2656],
         [ 13.0703, -17.9375, -17.5000,  ..., -16.6406, -18.0625, -17.3281],
         ...,
         [ 11.8359, -16.9219, -16.5312,  ..., -15.7188, -17.0000, -16.2969],
         [ 12.5781, -17.8594, -17.4688,  ..., -16.5938, -17.9844, -17.2500],
         [ 12.5000, -17.9688, -17.5156,  ..., -16.6406, -18.0625, -17.2969]]],
       device='cuda:0', grad_fn=<ToCopyBackward0>)
tensor([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 41,  8,  0,  0,  5,  0,  0,  0,
        41,  0,  0, 25,  0,  0,  5,  0,  0,  0, 49,  0,  0, 13,  0,  0, 23,  0,
         0, 41,  0,  0,  0,  0, 66,  0,  0, 36,  0,  0,  0,  0,  6,  0,  0,

# Push

In [ ]:
# save the checkpoint for inference
trainer.push_to_hub()

events.out.tfevents.1701631892.b81559cc4fba.4457.1:   0%|          | 0.00/406 [00:00<?, ?B/s]

'https://huggingface.co/Roxysun/cs2no-wav2vec2-large-xls-r-300m-cs-colab-phoneme/tree/main/'

# Test Fine-tuned model

In [ ]:
model = Wav2Vec2ForCTC.from_pretrained("Roxysun/"+repo_name).to("cuda")
# processor = Wav2Vec2Processor.from_pretrained(repo_name)
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-lv-60-espeak-cv-ft")

config.json:   0%|          | 0.00/2.34k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

In [ ]:
input_dict = processor(common_voice_test[0]["input_values"], return_tensors="pt", padding=True)
print(input_dict)

logits = model(input_dict.input_values.to("cuda")).logits
print(logits)

pred_ids = torch.argmax(logits, dim=-1)[0]
print(pred_ids)


print("Prediction:")
print(processor.decode(pred_ids))

print("\nReference:")
print(common_voice_test_2[0][sentence1].lower())

It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


{'input_values': tensor([[0.0132, 0.0069, 0.0082,  ..., 0.0195, 0.0217, 0.0263]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1]], dtype=torch.int32)}
tensor([[[ 12.8922, -17.9128, -17.5112,  ..., -16.5920, -18.0567, -17.2719],
         [ 12.9753, -17.8661, -17.4547,  ..., -16.5581, -18.0249, -17.2712],
         [ 13.0757, -17.9371, -17.5008,  ..., -16.6482, -18.0733, -17.3457],
         ...,
         [ 11.8407, -16.9272, -16.5326,  ..., -15.7172, -17.0059, -16.2994],
         [ 12.5816, -17.8734, -17.4683,  ..., -16.5933, -17.9854, -17.2598],
         [ 12.5033, -17.9699, -17.5140,  ..., -16.6437, -18.0621, -17.3127]]],
       device='cuda:0', grad_fn=<ViewBackward0>)
tensor([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 41,  8,  0,  0,  5,  0,  0,  0,
        41,  0,  0, 25,  0,  0,  5,  0,  0,  0, 49,  0,  0, 13,  0,  0, 23,  0,
         0, 41,  0,  0,  0,  0, 66,  0,  0, 36,  0,  0,  0,  0,  6,  0,  0,  